<a href="https://colab.research.google.com/github/MolecularFoundry/crucible-analysis-notebooks/blob/main/insitu-analysis/InSituUVvis_Fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib import ticker
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy import signal
from scipy.signal import argrelextrema

import copy
import os
import traceback


In [ ]:
# ================Functions=================================================
def decayGauss(x, a1, x1, w1, y0, a2, x2, t, a3):
    return y0 + a1 * np.exp(-(x - x1) ** 2 / w1) + a2 * np.exp(-(x - x2) * t) + a3 * x

def gauss(x, a1, x1, w1, y0):
    return y0 + a1 * np.exp(-(x - x1) ** 2 / w1)

def doubleGauss(x, a1, x1, w1, y0, a2, x2, w2, a3, x3, t, a4):
    return y0 + a1 * np.exp(-(x - x1) ** 2 / w1) + a2 * np.exp(-(x - x2) ** 2 / w2) + a3 * np.exp(-(x - x3) * t) + a4 * x

def tripleGauss(x, a1, x1, w1, y0, a2, x2, w2, a3, x3, w3):
    return y0 + a1 * np.exp(-(x - x1) ** 2 / w1) + a2 * np.exp(-(x - x2) ** 2 / w2) + a3 * np.exp(-(x - x3) ** 2 / w3)


In [ ]:
# mount your google drives
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
inputDict = {
              "energyScale" : False,
              "bkgCorr" : True,

              # Fit-parameters
              "estPeakPos" : 390,
              "estPeakPos2" : 320,
              "estPeakPos3" : 620,
              "estPeakWidth": 10
               }

### Load and View Sample

In [ ]:
project_id = "MFP08449 (1)"
sample_options = glob.glob(f"/content/drive/Shareddrives/{project_id}/Datasets/*/*_dataframe.csv")
print("Samples found at the path above are: ")
for s in sample_options:
  print(s)

In [ ]:
sample_path = "/content/drive/Shareddrives/MFP08449 (1)/Datasets/0sprqkja99vjh0005y13zmnxnw/USt_Raj_Ref_Annealing50_dataframe.csv" # copy path from output above
sample_name = os.path.basename(sample_path).split("_dataframe.csv")[0]

In [ ]:
df = pd.read_csv(f"{sample_path}", index_col = "wl")

num_sort = [str(x) for x in sorted([float(x) for x in df.columns])]
df = df.reindex(num_sort, axis=1)

# load and convert to arrays
df_y = np.array(df.index)
df_x = np.array([float(x) for x in df.columns])

df = df.to_numpy()
df = np.where(df <= 0, 0, df)
df = -np.log10(df / 100)


# transition to energy scale of the y axis
if inputDict['energyScale'] == True:
    df_y = [1240 / i for i in df_y]
    estPeakPos = 1240 / inputDict["estPeakPos"]
    estPeakPos2 = 1240 / inputDict["estPeakPos2"]
    estPeakPos3 = 1240 / inputDict["estPeakPos3"]
    estPeakWidth = inputDict["estPeakWidth"]

    # Jacobian transformation for all measured PL values (basically dividing by E^2)
    for i in range(np.shape(df)[1]):
        df[:, i] = df[:, i] / df_y / df_y

    # Mirroring dataframes to prevent sorting issues
    df_y = np.flip(df_y)
    df = np.flip(df, axis=0)

    yAxisLabel = 'Energy (eV)'

else:
    yAxisLabel = 'Wavelength (nm)'

# Simple plot of the data
plot_name = f"{sample_name}_2D_Plot.png"

plt.figure(1)
plt.contourf(df_x, df_y, df, 20, cmap=plt.cm.jet)

# Make a colorbar for the ContourSet
cbar = plt.colorbar()
cbar.ax.set_ylabel('Intensity (a.u.)')

# adding labels
plt.xlabel('Time (s)')
plt.ylabel(yAxisLabel)
plt.title(plot_name.strip(".png"))

plt.savefig(plot_name)

### (Optional) Run Background Subtraction

In [ ]:
# define the following in eV or nm ( you can pick a range on either side of the peak )

# background correction range 1 start/end value
y_bkgStart1 =0
y_bkgEnd1 = 1

# background correction range 2 start/end value
y_bkgStart2 = 3
y_bkgEnd2 = 5

In [ ]:
# run bkgd subtraction
l_bkgStart1 = next(xStart for xStart, valStart in enumerate(df_y) if valStart > y_bkgStart1)
l_bkgEnd1 = next(xEnd for xEnd, valEnd in enumerate(df_y) if valEnd > y_bkgEnd1) - 1
l_bkgStart2 = next(xStart for xStart, valStart in enumerate(df_y) if valStart > y_bkgStart2)
l_bkgEnd2 = next(xEnd for xEnd, valEnd in enumerate(df_y) if valEnd > y_bkgEnd2) - 1

for i in range(0, len(df_x)):
    xVals = np.concatenate([df_y[l_bkgStart1:l_bkgEnd1], df_y[l_bkgStart2:l_bkgEnd2]])
    yVals = np.concatenate([df[l_bkgStart1:l_bkgEnd1, i], df[l_bkgStart2:l_bkgEnd2, i]])
    coefs = np.polyfit(xVals, yVals, 1)  # The last parameter determines the order of the polynomial fit (0=const)
    poly1d_fn = np.poly1d(coefs)
    df[:, i] = df[:, i] - poly1d_fn(df_y[:])

### Plot Region of Interest

In [ ]:
# define the following
startTime = 0.0

# Wavelength/Energy region to be plotted
y_start = 300
y_end = 400

In [ ]:
# run
startTimeIdx = next(tStart for tStart, valStart in enumerate(df_x) if valStart > startTime)
l_start = next(xStart for xStart, valStart in enumerate(df_y) if valStart > y_start)
l_end = next(xEnd for xEnd, valEnd in enumerate(df_y) if valEnd > y_end) - 1

# shortening the wavelength/energy region to interesting region
df_cut = df[l_start:l_end+1, startTimeIdx:]

# same shortening for the Y-axis and X-axis arrays
df_yCut = df_y[l_start:l_end+1]
df_xCut = df_x[startTimeIdx:]
df_xCut = df_xCut - startTime


# Show the resulting data
plt.figure(2)
plt.contourf(df_xCut, df_yCut, df_cut, 20, cmap=plt.cm.jet)
# Make a colorbar for the ContourSet
cbar = plt.colorbar()
cbar.ax.set_ylabel('Intensity (a.u.)')
# adding labels
plt.xlabel('Time (s)')
plt.ylabel(yAxisLabel)
plt.title(str(sample_name) + ' _2D_Plot')
plt.pause(1)

### Saving the Results
Optimizing the data for plots in Igor

In [ ]:
# Optimizing data for plots in Igor
df_Igor = np.transpose(df_cut)
df_xIgor = np.insert(df_xCut, 0, 0)  # Adding a 0 as first element to accommodate for igor plot and link the timescale to zero for nice plots.
df_yIgor = np.append(df_yCut, df_yCut[-1]+df_yCut[-1]-df_yCut[-2])

In [ ]:
result_path = os.path.join(os.path.dirname(sample_path), "fits")
try:
  os.mkdir(result_path)
except Exception as e:
  if not isinstance(e, FileExistsError):
    print(type(e))


np.savetxt(f'{result_path}/{sample_name}_2D.csv', df_Igor, delimiter=",")
np.savetxt(f'{result_path}/{sample_name}_time.csv', df_xIgor, delimiter=",", header='Time_' + sample_name, comments='')
np.savetxt(f'{result_path}/{sample_name}_wavelength.csv', df_yIgor, delimiter=",", header='WL_' + sample_name, comments='')
np.savetxt(f'{result_path}/{sample_name}_LastSpectrum.csv', df_Igor[-1, :], delimiter=",", header='LastSpectrum_' + sample_name, comments='')

### Fit the Data